In [1]:
dataframe = spark.read.option("multiline", "true").json("Files/api_data.json")
display(dataframe)

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 763f9ef7-84d9-4fdf-9961-b02efc11cd42)

In [2]:
from pyspark.sql.functions import explode
import json
from pyspark.sql.types import StructType,StructField,StringType
from pyspark.sql.functions import to_date,date_format
from pyspark.sql.utils import AnalysisException

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 4, Finished, Available, Finished)

In [3]:
dataframe=dataframe.select("value")

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 5, Finished, Available, Finished)

In [4]:
expld = dataframe.select(explode(dataframe["value"]).alias("json_object"))

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 6, Finished, Available, Finished)

In [5]:
display(expld)

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 29f58a32-351e-45c7-91ca-9b07ed457009)

In [6]:
listofJson = expld.toJSON().collect()

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 8, Finished, Available, Finished)

In [7]:
print(listofJson[1])

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 9, Finished, Available, Finished)

{"json_object":{"about":[{"name":"Flavio Briatore","readLink":"https://api.bing.microsoft.com/api/v7/entities/149ab307-d658-f947-9d80-c0639d4a1084"},{"name":"Max Verstappen","readLink":"https://api.bing.microsoft.com/api/v7/entities/d2a3aa4d-dc3d-18cf-4d28-bab4616ad099"},{"name":"Lewis Hamilton","readLink":"https://api.bing.microsoft.com/api/v7/entities/afd8d17d-2960-d4d6-3ee3-cbb7ad7d515e"},{"name":"George Russell","readLink":"https://api.bing.microsoft.com/api/v7/entities/c131b56c-d34b-019c-28bf-b1cb184cd721"},{"name":"Ferrari","readLink":"https://api.bing.microsoft.com/api/v7/entities/5578729a-73af-5f1a-9b3c-e468b44acfb4"},{"name":"Brackley","readLink":"https://api.bing.microsoft.com/api/v7/entities/f709e9a0-1ae8-b34e-947d-31284a0eb096"},{"name":"Bologna","readLink":"https://api.bing.microsoft.com/api/v7/entities/f36c4093-c958-bcb0-7345-a64ab939999b"},{"name":"Toto Wolff","readLink":"https://api.bing.microsoft.com/api/v7/entities/58d54a50-d7f6-6ab4-a99b-a7f9c8bae23e"},{"name":"Formu

In [8]:
dataofJson = json.loads(listofJson[1])

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 10, Finished, Available, Finished)

In [9]:
# Initialize lists
article_title = []
article_description = []
article_category = []
article_url = []
article_image = []
article_provider = []
article_date_published = []

for item in listofJson:
    try:
        # Parse the object
        data = json.loads(item)
        json_obj = data.get("json_object", {})

        # Check if required fields exist before accessing them
        if json_obj.get("category") and json_obj.get("provider", [{}])[0].get("image", {}).get("thumbnail", {}).get("contentUrl"):
            article_title.append(json_obj.get("name", ""))
            article_description.append(json_obj.get("description", ""))
            article_category.append(json_obj.get("category", ""))
            article_url.append(json_obj.get("url", ""))
            article_image.append(json_obj["provider"][0]["image"]["thumbnail"].get("contentUrl", ""))
            article_provider.append(json_obj["provider"][0].get("name", ""))
            article_date_published.append(json_obj.get("datePublished", ""))

    except Exception as exc:
        print(f"Unable to process this JSON object due to: {exc}")


StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 11, Finished, Available, Finished)

In [10]:
# Combine data 
info = list(
    zip(
        article_title,
        article_description,
        article_category,
        article_url,
        article_image,
        article_provider,
        article_date_published
    )
)

# Define the schema 
framework = StructType([
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("url", StringType(), True),
    StructField("image", StringType(), True),
    StructField("provider", StringType(), True),
    StructField("datePublished", StringType(), True)
])

# Create the DataFrame 
cleaned_dataframe = spark.createDataFrame(info, schema=framework)


StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 12, Finished, Available, Finished)

In [11]:
display(cleaned_dataframe.limit(5))

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d6992f94-d610-4258-b6eb-22b3d86c202e)

In [12]:
finaldataframe = cleaned_dataframe.withColumn("datePublished",date_format(to_date("datePublished"),"dd-MMM-yyyy"))

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 14, Finished, Available, Finished)

In [13]:
display(finaldataframe.limit(5))

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 834783c8-4e25-4f18-bde6-a3f4681df9ae)

In [14]:
try:
    tabletag = 'bing_api_db.tbl_API_news'
    finaldataframe.write.format("delta").saveAsTable(tabletag)

except AnalysisException:

    print(" Table already available")

    finaldataframe.createOrReplaceTempView("vw_finaldataframe")

    spark.sql(f"""  MERGE INTO {tabletag} aimedtable
                    USING vw_finaldataframe originview
                    ON originview.url = aimedtable.url
                    WHEN MATCHED AND
                    originview.title <> aimedtable.title OR 
                    originview.description <> aimedtable.description OR
                    originview.category <> aimedtable.category OR 
                    originview.image <> aimedtable.image OR
                    originview.provider <> aimedtable.provider OR 
                    originview.datePublished <> aimedtable.datePublished 

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *
                    
                """)

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 16, Finished, Available, Finished)

 Table already available


In [15]:
%%sql
select count(*) from bing_api_db.tbl_API_news

StatementMeta(, a6a76833-2e95-4794-9342-a8365a66a591, 17, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>